In [1]:
from transformers import pipeline
import pandas as pd # panda can store tables

In [2]:
!pip install langchain transformers torch accelerate

In [3]:
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "Enter your own HuggingFace access token here"

In [4]:
!pip install -U langchain langchain-community

In [4]:
!pip install "numpy<2"

In [5]:
import torch
from transformers import pipeline # pipeline contains all the trained DL models
from langchain.llms import HuggingFacePipeline
from langchain import HuggingFaceHub
# llm=HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":1, "max_length":512})

In [6]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [7]:
# explicitly specify the model
model_name = "google/flan-t5-large"

# Create hugging face pipeline
hf_pipeline = pipeline("text2text-generation", model = model_name)

# wrap the pipeline inside langchain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline = hf_pipeline)

Device set to use cpu


In [8]:
template = """ Question: {question}
Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [9]:
# Question 1
question = "What is the capital of France?"
print(llm_chain.run(question))

/tmp/ipython-input-2201930997.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain.run(question))


paris


In [10]:
# Question 2
question = "Explain the concept of black hole in simple terms?"
print(llm_chain.run(question))

A black hole is a region of space that is not visible to the naked eye .


In [11]:
# Question 3
question = "how can we address global warming?"
print(llm_chain.run(question))

reducing the use of fossil fuels


In [12]:
from transformers import pipeline
from langchain import HuggingFacePipeline
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate

# Step 0: Define LLM for answering (English generation)
gen_model = "google/flan-t5-large"
gen_pipeline = pipeline("text2text-generation", model=gen_model)
answer_llm = HuggingFacePipeline(pipeline=gen_pipeline)

# Step 1: Define LLM for translation (English → Hindi)
trans_model = "Helsinki-NLP/opus-mt-en-hi"
trans_pipeline = pipeline("translation", model=trans_model)
trans_llm = HuggingFacePipeline(pipeline=trans_pipeline)

# Step 2: Answer generation
gen_prompt = PromptTemplate.from_template("Answer the question: {question}")
gen_chain = LLMChain(llm=answer_llm, prompt=gen_prompt, output_key="answer")

# Step 3: Translation
translate_prompt = PromptTemplate.from_template("{answer}")  # just pass text
translate_chain = LLMChain(llm=trans_llm, prompt=translate_prompt, output_key="answer-hindi")

# Step 4: Combine
overall_chain = SequentialChain(
    chains=[gen_chain, translate_chain],
    input_variables=["question"],
    output_variables=["answer", "answer-hindi"]
)

# Example usage
res = overall_chain({"question": "Explain the concept of black hole in simple terms?"})
print("Answer in English:", res["answer"])
print("Answer in Hindi:", res["answer-hindi"])

Device set to use cpu
/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


Answer in English: A black hole is a region of space that is not visible to the naked eye .
Answer in Hindi: एक ब्लैक होल अंतरिक्ष का क्षेत्र है जो नंगी आँख से दिखाई देता है ।
